In [3]:
import os

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama.llms import OllamaLLM
from langchain_huggingface import HuggingFaceEmbeddings

# Define the directory containing the text file and the persistent directory
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "books", "odyssey.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db")

# Check if the Chroma vector store already exists
if not os.path.exists(persistent_directory):
    print("Persistent directory does not exist. Initializing vector store...")

    # Ensure the text file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(
            f"The file {file_path} does not exist. Please check the path."
        )

    # Read the text content from the file
    loader = TextLoader(file_path)
    documents = loader.load()

    # Split the document into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # Display information about the split documents
    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(docs)}")
    print(f"Sample chunk:\n{docs[0].page_content}\n")

    # Create embeddings
    print("\n--- Creating embeddings ---")
    embeddings = "sentence-transformers/all-mpnet-base-v2"
    # Update to a valid embedding model if needed
    print("\n--- Finished creating embeddings ---")

    # Create the vector store and persist it automatically
    print("\n--- Creating vector store ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory)
    print("\n--- Finished creating vector store ---")

else:
    print("Vector store already exists. No need to initialize.")

Vector store already exists. No need to initialize.


In [7]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embedding_function)

# Retriever 설정
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.9},
)

# 질의 실행
query = "What happened to Odysseus after Troy?"
relevant_docs = retriever.invoke(query)


/home/changeroa/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No relevant docs were retrieved using the relevance score threshold 0.9


In [9]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM

# 디렉토리 설정
current_dir = os.getcwd()
books_dir = os.path.join(current_dir, "books")
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata")

print(f"Books directory: {books_dir}")
print(f"Persistent directory: {persistent_directory}")

# 벡터 스토어 초기화 여부 확인
if not os.path.exists(persistent_directory):
    print("Persistent directory does not exist. Initializing vector store...")

    if not os.path.exists(books_dir):
        raise FileNotFoundError(
            f"The directory {books_dir} does not exist. Please check the path."
        )

    # .txt 파일 목록
    book_files = [f for f in os.listdir(books_dir) if f.endswith(".txt")]

    # 문서 로드 및 메타데이터 추가
    documents = []
    for book_file in book_files:
        file_path = os.path.join(books_dir, book_file)
        loader = TextLoader(file_path)
        book_docs = loader.load()
        for doc in book_docs:
            doc.metadata = {"source": book_file}
            documents.append(doc)

    # 문서 분할
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    print("\n--- Document Chunks Information ---")
    print(f"Number of document chunks: {len(docs)}")

    # HuggingFace 임베딩 생성
    print("\n--- Creating embeddings ---")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2"
    )
    print("\n--- Finished creating embeddings ---")

    # 벡터 스토어 생성 및 저장
    print("\n--- Creating and persisting vector store ---")
    db = Chroma.from_documents(
        docs, embeddings, persist_directory=persistent_directory
    )
    print("\n--- Finished creating and persisting vector store ---")

else:
    print("Vector store already exists. No need to initialize.")


Books directory: /home/changeroa/projects/Agents/langchain-crash-course-main/4_rag/books
Persistent directory: /home/changeroa/projects/Agents/langchain-crash-course-main/4_rag/db/chroma_db_with_metadata
Vector store already exists. No need to initialize.


In [10]:
import os
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Define the persistent directory
current_dir = os.getcwd()
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata")

# Define the embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Load the existing vector store with the embedding function
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# Define the user's question
query = "How did Juliet die?"

# Retrieve relevant documents based on the query
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.1},
)
relevant_docs = retriever.invoke(query)

# Display the relevant results with metadata
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


No relevant docs were retrieved using the relevance score threshold 0.1



--- Relevant Documents ---


In [12]:
import os
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TextSplitter,
    TokenTextSplitter,
)
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# 디렉토리 설정
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "books", "romeo_and_juliet.txt")
db_dir = os.path.join(current_dir, "db")

# 텍스트 파일 존재 여부 확인
if not os.path.exists(file_path):
    raise FileNotFoundError(f"The file {file_path} does not exist. Please check the path.")

# 문서 로드
loader = TextLoader(file_path)
documents = loader.load()

# 임베딩 모델 정의
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# 벡터스토어 생성 함수
def create_vector_store(docs, store_name):
    persistent_directory = os.path.join(db_dir, store_name)
    if not os.path.exists(persistent_directory):
        print(f"\n--- Creating vector store {store_name} ---")
        db = Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory
        )
        print(f"--- Finished creating vector store {store_name} ---")
    else:
        print(f"Vector store {store_name} already exists. No need to initialize.")

# 1. Character-based Splitting
print("\n--- Using Character-based Splitting ---")
char_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
char_docs = char_splitter.split_documents(documents)
create_vector_store(char_docs, "chroma_db_char")

# 2. Sentence-based Splitting
print("\n--- Using Sentence-based Splitting ---")
sent_splitter = SentenceTransformersTokenTextSplitter(chunk_size=1000)
sent_docs = sent_splitter.split_documents(documents)
create_vector_store(sent_docs, "chroma_db_sent")

# 3. Token-based Splitting
print("\n--- Using Token-based Splitting ---")
token_splitter = TokenTextSplitter(chunk_overlap=0, chunk_size=512)
token_docs = token_splitter.split_documents(documents)
create_vector_store(token_docs, "chroma_db_token")

# 4. Recursive Character-based Splitting
print("\n--- Using Recursive Character-based Splitting ---")
rec_char_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
rec_char_docs = rec_char_splitter.split_documents(documents)
create_vector_store(rec_char_docs, "chroma_db_rec_char")

# 5. Custom Splitting
print("\n--- Using Custom Splitting ---")
class CustomTextSplitter(TextSplitter):
    def split_text(self, text):
        return text.split("\n\n")  # 단락 기준

custom_splitter = CustomTextSplitter()
custom_docs = custom_splitter.split_documents(documents)
create_vector_store(custom_docs, "chroma_db_custom")

# 질의 함수 정의
def query_vector_store(store_name, query):
    persistent_directory = os.path.join(db_dir, store_name)
    if os.path.exists(persistent_directory):
        print(f"\n--- Querying the Vector Store {store_name} ---")
        db = Chroma(
            persist_directory=persistent_directory, embedding_function=embeddings
        )
        retriever = db.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"k": 1, "score_threshold": 0.1},
        )
        relevant_docs = retriever.invoke(query)
        print(f"\n--- Relevant Documents for {store_name} ---")
        for i, doc in enumerate(relevant_docs, 1):
            print(f"Document {i}:\n{doc.page_content}\n")
            print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")
    else:
        print(f"Vector store {store_name} does not exist.")

# 사용자 질의 정의
query = "How did Juliet die?"

# 각 벡터스토어에 질의 실행
query_vector_store("chroma_db_char", query)
query_vector_store("chroma_db_sent", query)
query_vector_store("chroma_db_token", query)
query_vector_store("chroma_db_rec_char", query)
query_vector_store("chroma_db_custom", query)


Created a chunk of size 1105, which is longer than the specified 1000
Created a chunk of size 1437, which is longer than the specified 1000
Created a chunk of size 1871, which is longer than the specified 1000
Created a chunk of size 1015, which is longer than the specified 1000
Created a chunk of size 1006, which is longer than the specified 1000
Created a chunk of size 1054, which is longer than the specified 1000
Created a chunk of size 1432, which is longer than the specified 1000
Created a chunk of size 1367, which is longer than the specified 1000
Created a chunk of size 2178, which is longer than the specified 1000
Created a chunk of size 1390, which is longer than the specified 1000
Created a chunk of size 1502, which is longer than the specified 1000
Created a chunk of size 1410, which is longer than the specified 1000
Created a chunk of size 1741, which is longer than the specified 1000
Created a chunk of size 1184, which is longer than the specified 1000
Created a chunk of s


--- Using Character-based Splitting ---

--- Creating vector store chroma_db_char ---
--- Finished creating vector store chroma_db_char ---

--- Using Sentence-based Splitting ---

--- Creating vector store chroma_db_sent ---
--- Finished creating vector store chroma_db_sent ---

--- Using Token-based Splitting ---

--- Creating vector store chroma_db_token ---
--- Finished creating vector store chroma_db_token ---

--- Using Recursive Character-based Splitting ---

--- Creating vector store chroma_db_rec_char ---
--- Finished creating vector store chroma_db_rec_char ---

--- Using Custom Splitting ---

--- Creating vector store chroma_db_custom ---
--- Finished creating vector store chroma_db_custom ---

--- Querying the Vector Store chroma_db_char ---

--- Relevant Documents for chroma_db_char ---
Document 1:
PARIS.
O, I am slain! [_Falls._] If thou be merciful,
Open the tomb, lay me with Juliet.

 [_Dies._]

ROMEO.
In faith, I will. Let me peruse this face.
Mercutio’s kinsman, nobl

In [1]:
import os
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


# Define the persistent directory
current_dir = os.getcwd()  # Jupyter 환경 호환
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "chroma_db_apple")

# Step 1: Scrape content from apple.com
urls = ["https://www.apple.com/"]
loader = WebBaseLoader(urls)
documents = loader.load()

# Step 2: Split the scraped content into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

print("\n--- Document Chunks Information ---")
print(f"Number of document chunks: {len(docs)}")
print(f"Sample chunk:\n{docs[0].page_content}\n")

# Step 3: Use HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Step 4: Create or load the vector store
if not os.path.exists(persistent_directory):
    print(f"\n--- Creating vector store in {persistent_directory} ---")
    db = Chroma.from_documents(docs, embeddings, persist_directory=persistent_directory)
    print(f"--- Finished creating vector store in {persistent_directory} ---")
else:
    print(f"Vector store {persistent_directory} already exists. Loading existing store.")
    db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)

# Step 5: Query the vector store
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

query = "What new products are announced on Apple.com?"
relevant_docs = retriever.invoke(query)

# Display results
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


USER_AGENT environment variable not set, consider setting it to identify your requests.
Created a chunk of size 1070, which is longer than the specified 1000
Created a chunk of size 1305, which is longer than the specified 1000



--- Document Chunks Information ---
Number of document chunks: 7
Sample chunk:
Apple


Apple

AppleStoreMaciPadiPhoneWatch
VisionAirPodsTV & HomeEntertainmentAccessoriesSupport


0+

 

iPhone 16 Pro
Hello, Apple Intelligence.

Learn more
Buy


 

iPhone 16
Hello, Apple Intelligence.

Learn more
Buy


 

Apple Watch Series 10
Thinstant classic.

Learn more
Buy


 

Apple Watch Ultra 2
New finish. Never quit.

Learn more
Buy


 

Apple Watch
Live healthier. Train better. Stay connected.

Learn more
Shop Watch


 


WWDC 25

Apple Worldwide Developers Conference. Join us online June 9–13.

Learn more


 


MacBook Air
Sky blue color.Sky high performance with M4.

Learn more
Buy

Built for Apple Intelligence.


 


iPad Air
Now supercharged by the M3 chip.

Learn more
Buy

Built for Apple Intelligence.


 

AirPods Pro 2
Now with a Hearing Aid feature.1


Learn more
Buy


 

Apple Trade In
Get $170–$630 in credit when you trade in iPhone 12 or higher.2

Get your estimate


 

Apple Card


/home/changeroa/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



--- Creating vector store in /home/changeroa/projects/Agents/langchain-crash-course-main/4_rag/db/chroma_db_apple ---
--- Finished creating vector store in /home/changeroa/projects/Agents/langchain-crash-course-main/4_rag/db/chroma_db_apple ---

--- Relevant Documents ---
Document 1:
Apple


Apple

AppleStoreMaciPadiPhoneWatch
VisionAirPodsTV & HomeEntertainmentAccessoriesSupport


0+

 

iPhone 16 Pro
Hello, Apple Intelligence.

Learn more
Buy


 

iPhone 16
Hello, Apple Intelligence.

Learn more
Buy


 

Apple Watch Series 10
Thinstant classic.

Learn more
Buy


 

Apple Watch Ultra 2
New finish. Never quit.

Learn more
Buy


 

Apple Watch
Live healthier. Train better. Stay connected.

Learn more
Shop Watch


 


WWDC 25

Apple Worldwide Developers Conference. Join us online June 9–13.

Learn more


 


MacBook Air
Sky blue color.Sky high performance with M4.

Learn more
Buy

Built for Apple Intelligence.


 


iPad Air
Now supercharged by the M3 chip.

Learn more
Buy

Built for Ap